<a href="https://colab.research.google.com/github/CodewithTanzeel/OpenAi-Agents-SDK/blob/main/OpenAiagentsSDK3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai-agents
# Optional: for voice / realtime agents
# !pip install 'openai-agents[voice]'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.4/161.4 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.5/132.5 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.5/158.5 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.8 MB/s eta 0:00:00


In [14]:
import os
import asyncio
import nest_asyncio

from agents import Agent, Runner, AsyncOpenAI, OpenAIChatCompletionsModel,SQLiteSession
from agents.run import RunConfig

nest_asyncio.apply()


In [15]:
from google.colab import userdata
gemini_api_key = userdata.get("GEMINI_API_KEY")

In [16]:
external_client:AsyncOpenAI = AsyncOpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

model:OpenAIChatCompletionsModel = OpenAIChatCompletionsModel(
    model="gemini-2.5-flash",
    openai_client=external_client
)

# Basic Agent + Session **Flow**

In [29]:
# Define a simple agent
agent:Agent = Agent(
    name="Assistant",
    instructions="Reply very concisely.",
     model=model
)

# Create a session: each session ID stores conversation history
session = SQLiteSession("conversation_123")

# First interaction
result:Runner = Runner.run_sync(agent, "What city is Golden bridge in.",session=session)
print("Assistant:", result.final_output)

# Second interaction: session auto-remembers the previous turn
result2 = Runner.run_sync(agent, "What state is it in?", session=session)
print("Assistant:", result2.final_output)  # Should say “California”

Assistant: San Francisco
Assistant: California


##**Memory Operations**

In [30]:
# Use async to demonstrate memory operations
import asyncio
from agents.memory import SQLiteSession as AsyncSQLiteSession

async def memory_demo():
    session = AsyncSQLiteSession("demo_session")
    await Runner.run(agent, "Hello there!", session=session)
    await Runner.run(agent, "Who are you?", session=session)

    items = await session.get_items()
    print("All session items:", items)

    last = await session.pop_item()
    print("Popped last:", last)

    await session.clear_session()
    print("Session cleared:", await session.get_items())

asyncio.run(memory_demo())


All session items: [{'content': 'Hello there!', 'role': 'user'}, {'id': '__fake_id__', 'content': [{'annotations': [], 'text': 'Hi.', 'type': 'output_text'}], 'role': 'assistant', 'status': 'completed', 'type': 'message'}, {'content': 'Who are you?', 'role': 'user'}, {'id': '__fake_id__', 'content': [{'annotations': [], 'text': 'I am a large language model, trained by Google.', 'type': 'output_text'}], 'role': 'assistant', 'status': 'completed', 'type': 'message'}]
Popped last: {'id': '__fake_id__', 'content': [{'annotations': [], 'text': 'I am a large language model, trained by Google.', 'type': 'output_text'}], 'role': 'assistant', 'status': 'completed', 'type': 'message'}
Session cleared: []


## **Session Options**

In [31]:
Runner.run_sync(agent, "Hi!")  # Stateless—does NOT remember previous context


RunResult(input='Hi!', new_items=[MessageOutputItem(agent=Agent(name='Assistant', handoff_description=None, tools=[], mcp_servers=[], mcp_config={}, instructions='Reply very concisely.', prompt=None, handoffs=[], model=<agents.models.openai_chatcompletions.OpenAIChatCompletionsModel object at 0x786aeb8d1f90>, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, response_include=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True), raw_item=ResponseOutputMessage(id='__fake_id__', content=[ResponseOutputText(annotations=[], text='Hi!', type='output_text', logprobs=None)], role='assistant', status='completed', type='message'), type='message_ou

In [33]:
session = SQLiteSession("user_id", "conversations.db")

## **Custom Session Implementation**

In [34]:
from agents.memory import Session

class MySession(Session):
    def __init__(self, session_id):
        self.session_id = session_id
        self._items = []

    async def get_items(self, limit=None):
        return self._items[-limit:] if limit else self._items

    async def add_items(self, items):
        self._items.extend(items)

    async def pop_item(self):
        return self._items.pop() if self._items else None

    async def clear_session(self):
        self._items.clear()


In [35]:
session = MySession("custom1")
Runner.run_sync(agent, "Hi again!", session=session)


RunResult(input=[{'content': 'Hi again!', 'role': 'user'}], new_items=[MessageOutputItem(agent=Agent(name='Assistant', handoff_description=None, tools=[], mcp_servers=[], mcp_config={}, instructions='Reply very concisely.', prompt=None, handoffs=[], model=<agents.models.openai_chatcompletions.OpenAIChatCompletionsModel object at 0x786aeb8d1f90>, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, response_include=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True), raw_item=ResponseOutputMessage(id='__fake_id__', content=[ResponseOutputText(annotations=[], text='Hi!', type='output_text', logprobs=None)], role='assistant', status='complete